# Analysis of Household Water Usage

## 0. Load required libraries

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import seaborn as sns
import plotly.graph_objects as go


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
ROOT = "/content/drive/MyDrive/Group8"

## 1. Load data

In [ ]:
aggregated_df = pd.read_csv(f"{ROOT}/data/raw/aggregatedWholeHouse.csv")
Bidet_df = pd.read_csv(f"{ROOT}/data/processed/Bidet_interpolated.csv")
Kitchen_faucet_df = pd.read_csv(f"{ROOT}/data/processed/Kitchen_interpolated.csv")
Shower_df = pd.read_csv(f"{ROOT}/data/processed/Shower_interpolated.csv")
Washbasin_df = pd.read_csv(f"{ROOT}/data/raw/feedWashbasin.csv")
Washing_Machine_df = pd.read_csv(f"{ROOT}/data/processed/Washing Machine_interpolated.csv")
Dishwasher_df = pd.read_csv(f"{ROOT}/data/raw/feedDishwasher.csv")
Toilet_df = pd.read_csv(f"{ROOT}/data/raw/feedToilet.csv")

## 2. Data Processing

In [ ]:
Bidet_df = Bidet_df.rename(columns={"bidetFlow": "Flow", "Unnamed: 0": "datetime"})
Kitchen_faucet_df = Kitchen_faucet_df.rename(columns={"kitchenFlow": "Flow", "Unnamed: 0": "datetime"})
Shower_df = Shower_df.rename(columns={"showerFlow": "Flow", "Unnamed: 0": "datetime"})
Washing_Machine_df = Washing_Machine_df.rename(columns={"machineFlow": "Flow", "Unnamed: 0": "datetime"})

In [ ]:
aggregated_df['datetime'] = pd.to_datetime(aggregated_df['unix'], unit='s')
aggregated_df.set_index('datetime', inplace=True)

Bidet_df['datetime'] = pd.to_datetime(Bidet_df['datetime'])
Bidet_df.set_index('datetime', inplace=True)

Kitchen_faucet_df['datetime'] = pd.to_datetime(Kitchen_faucet_df['datetime'])
Kitchen_faucet_df.set_index('datetime', inplace=True)

Shower_df['datetime'] = pd.to_datetime(Shower_df['datetime'])
Shower_df.set_index('datetime', inplace=True)

Washing_Machine_df['datetime'] = pd.to_datetime(Washing_Machine_df['datetime'])
Washing_Machine_df.set_index('datetime', inplace=True)

Washbasin_df['datetime'] = pd.to_datetime(Washbasin_df['Time'], unit='s')
Washbasin_df.set_index('datetime', inplace=True)

Dishwasher_df['datetime'] = pd.to_datetime(Dishwasher_df['Time'], unit='s')
Dishwasher_df['end_datetime'] = pd.to_datetime(Dishwasher_df['EndTime'], unit='s')
Dishwasher_df.set_index('datetime', inplace=True)

Toilet_df['datetime'] = pd.to_datetime(Toilet_df['Time'], unit='s')
Toilet_df['end_datetime'] = pd.to_datetime(Toilet_df['EndFlow'], unit='s')
Toilet_df.set_index('datetime', inplace=True)

In [ ]:
monthly_aggregated_df = aggregated_df.resample('M').sum()
monthly_Bidet_df = Bidet_df.resample("M").sum()
monthly_Kitchen_faucet_df = Kitchen_faucet_df.resample("M").sum()
monthly_Shower_df = Shower_df.resample("M").sum()
monthly_Washbasin_df = Washbasin_df.resample("M").sum()
monthly_Washing_Machine_df = Washing_Machine_df.resample("M").sum()
monthly_Dishwasher_df = Dishwasher_df.resample("M").sum()
monthly_Toilet_df = Toilet_df.resample("M").sum()

<ipython-input-7-009f5c42ec2c>:7: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_Dishwasher_df = Dishwasher_df.resample("M").sum()
<ipython-input-7-009f5c42ec2c>:8: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  monthly_Toilet_df = Toilet_df.resample("M").sum()


In [ ]:
monthly_Washbasin_df = monthly_Washbasin_df[monthly_Washbasin_df['Time'] > 0]

In [ ]:
Dishwasher_df.drop(['Time', 'EndTime'], axis=1, inplace=True)
Toilet_df.drop(['Time', 'EndFlow'], axis=1, inplace=True)

## 3. Analysis of Whole House: Aggregated data

In [ ]:
aggregated_df.head(5)

,unix,flow
datetime,,
2019-02-13 08:57:09,1550048229,0.0
2019-02-13 08:59:32,1550048372,177.0
2019-02-13 09:03:32,1550048612,0.0
2019-02-13 09:08:32,1550048912,0.0
2019-02-13 09:13:32,1550049212,0.0


In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_aggregated_df.index, y=monthly_aggregated_df['flow'], mode='lines+markers', name='Whole House'))

# Customize the plot layout
fig.update_layout(
    title='Monthly Flow Over Time of Whole House: Aggregated',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

fig.add_trace(go.Scatter(x=["2019-10-31"], y=[2.421700e+06], mode='markers', marker=dict(color='red', size=15), showlegend=False))
fig.add_trace(go.Scatter(x=["2020-09-30"], y=[2.795862e+06], mode='markers', marker=dict(color='red', size=15), showlegend=False))


fig.add_annotation(x="2019-10-31", y=2.421700e+06, text="Oct 31, 2019", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)
fig.add_annotation(x="2020-09-30", y=2.795862e+06, text="Sep 30, 2020", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

# Show the plot
fig.show()

## 4. Analysis of Individual Fixtures

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Bidet_df.index, y=monthly_Bidet_df['Flow'], mode='lines+markers', name='Bidet'))
fig.add_trace(go.Scatter(x=monthly_Kitchen_faucet_df.index, y=monthly_Kitchen_faucet_df['Flow'], mode='lines+markers', name='Kitchen Faucet'))
fig.add_trace(go.Scatter(x=monthly_Shower_df.index, y=monthly_Shower_df['Flow'], mode='lines+markers', name='Shower'))
fig.add_trace(go.Scatter(x=monthly_Washbasin_df.index, y=monthly_Washbasin_df['Flow'], mode='lines+markers', name='Washbasin'))
fig.add_trace(go.Scatter(x=monthly_Washing_Machine_df.index, y=monthly_Washing_Machine_df['Flow'], mode='lines+markers', name='Washing Machine'))
fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))

# Customize the plot layout
fig.update_layout(
    title='Flow Trend Of Individual Fixtures Over Time',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

## 5. Analysis of Shower

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Bidet_df.index, y=monthly_Bidet_df['Flow'], mode='lines+markers', name='Bidet'))
fig.add_trace(go.Scatter(x=monthly_Kitchen_faucet_df.index, y=monthly_Kitchen_faucet_df['Flow'], mode='lines+markers', name='Kitchen Faucet'))
fig.add_trace(go.Scatter(x=monthly_Shower_df.index, y=monthly_Shower_df['Flow'], mode='lines+markers', name='Shower', line=dict(width=4)))
fig.add_trace(go.Scatter(x=monthly_Washbasin_df.index, y=monthly_Washbasin_df['Flow'], mode='lines+markers', name='Washbasin'))
fig.add_trace(go.Scatter(x=monthly_Washing_Machine_df.index, y=monthly_Washing_Machine_df['Flow'], mode='lines+markers', name='Washing Machine'))
fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))

# Customize the plot layout
fig.update_layout(
    title='Shower shows maximum Flow Over Time',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Bidet_df.index, y=monthly_Bidet_df['Flow'], mode='lines+markers', name='Bidet'))
fig.add_trace(go.Scatter(x=monthly_Kitchen_faucet_df.index, y=monthly_Kitchen_faucet_df['Flow'], mode='lines+markers', name='Kitchen Faucet'))
fig.add_trace(go.Scatter(x=monthly_Shower_df.index, y=monthly_Shower_df['Flow'], mode='lines+markers', name='Shower', line=dict(width=4)))
fig.add_trace(go.Scatter(x=monthly_Washbasin_df.index, y=monthly_Washbasin_df['Flow'], mode='lines+markers', name='Washbasin'))
fig.add_trace(go.Scatter(x=monthly_Washing_Machine_df.index, y=monthly_Washing_Machine_df['Flow'], mode='lines+markers', name='Washing Machine'))
fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))

fig.add_trace(go.Scatter(x=[monthly_Shower_df.index[8]], y=[monthly_Shower_df['Flow'][8]], mode='markers', marker=dict(color='red', size=15), showlegend=False))
fig.add_trace(go.Scatter(x=[monthly_Shower_df.index[19]], y=[monthly_Shower_df['Flow'][19]], mode='markers', marker=dict(color='red', size=15), showlegend=False))

# Add a horizontal line
fig.add_trace(go.Scatter(x=[monthly_Shower_df.index[8], monthly_Shower_df.index[19]], y=[monthly_Shower_df['Flow'][8], monthly_Shower_df['Flow'][19]],
                         mode='lines', marker=dict(color='red', size=15), showlegend=False))

# Add text annotation
fig.add_annotation(x=monthly_Shower_df.index[8], y=monthly_Shower_df['Flow'][8], text=f"Oct 31, 2019", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

fig.add_annotation(x=monthly_Shower_df.index[19], y=monthly_Shower_df['Flow'][19], text=f"Sep 30, 2020", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

fig.add_annotation(x=monthly_Shower_df.index[13], y=monthly_Shower_df['Flow'][19] - monthly_Shower_df['Flow'][19]*0.1, text=f"{round(np.abs(((monthly_Shower_df['Flow'][8] - monthly_Shower_df['Flow'][19])/monthly_Shower_df['Flow'][8]*100)), 2)}%", showarrow=False,
                   arrowhead=1, arrowcolor="black", arrowsize=2, font=dict(size=15, color="red"), textangle=-10)

# Customize the plot layout
fig.update_layout(
    title='Analysis of Shower Annually',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

## 6. Comparing Shower with Other Fixtures

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Bidet_df.index, y=monthly_Bidet_df['Flow'], mode='lines+markers', name='Bidet'))
fig.add_trace(go.Scatter(x=monthly_Kitchen_faucet_df.index, y=monthly_Kitchen_faucet_df['Flow'], mode='lines+markers', name='Kitchen Faucet'))
fig.add_trace(go.Scatter(x=monthly_Shower_df.index, y=monthly_Shower_df['Flow'], mode='lines+markers', name='Shower', line=dict(width=4)))
fig.add_trace(go.Scatter(x=monthly_Washbasin_df.index, y=monthly_Washbasin_df['Flow'], mode='lines+markers', name='Washbasin'))
fig.add_trace(go.Scatter(x=monthly_Washing_Machine_df.index, y=monthly_Washing_Machine_df['Flow'], mode='lines+markers', name='Washing Machine'))
fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))

fig.add_trace(go.Scatter(x=[monthly_Shower_df.index[8]], y=[monthly_Shower_df['Flow'][8]], mode='markers', marker=dict(color='red', size=15), showlegend=False))
fig.add_trace(go.Scatter(x=[monthly_Shower_df.index[19]], y=[monthly_Shower_df['Flow'][19]], mode='markers', marker=dict(color='red', size=15), showlegend=False))

fig.add_trace(go.Scatter(x=[monthly_Washing_Machine_df.index[1]], y=[monthly_Washing_Machine_df['Flow'][1]], mode='markers', marker=dict(color='red', size=15), showlegend=False))
fig.add_trace(go.Scatter(x=[monthly_Kitchen_faucet_df.index[-3]], y=[monthly_Kitchen_faucet_df['Flow'][-3]], mode='markers', marker=dict(color='red', size=15), showlegend=False))

fig.add_shape(
    type="line",
    x0=monthly_Washing_Machine_df.index[1], y0=monthly_Shower_df['Flow'][8],
    x1=monthly_Washing_Machine_df.index[1], y1=monthly_Washing_Machine_df['Flow'][1],
    line=dict(color="red", width=2, dash="dashdot"),
)

fig.add_shape(
    type="line",
    x0=monthly_Kitchen_faucet_df.index[-3], y0=monthly_Shower_df['Flow'][19],
    x1=monthly_Kitchen_faucet_df.index[-3], y1=monthly_Kitchen_faucet_df['Flow'][-3],
    line=dict(color="red", width=2, dash="dashdot"),
)

# Add text annotation
fig.add_annotation(x=monthly_Shower_df.index[8], y=monthly_Shower_df['Flow'][8], text=f"Oct 31, 2019", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

fig.add_annotation(x=monthly_Shower_df.index[19], y=monthly_Shower_df['Flow'][19], text=f"Sep 30, 2020", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

# Fixtures
fig.add_annotation(x=monthly_Shower_df.index[9], y=monthly_Washing_Machine_df['Flow'][1] + monthly_Washing_Machine_df['Flow'][1] * 0.2, text=f"Washing Machine", showarrow=False,
                   arrowhead=1, arrowcolor="black", arrowsize=2, font=dict(size=13))

fig.add_annotation(x=monthly_Shower_df.index[18], y=monthly_Kitchen_faucet_df['Flow'][-3] + monthly_Kitchen_faucet_df['Flow'][-3] * 0.1, text=f"Kitchen Faucet", showarrow=False,
                   arrowhead=1, arrowcolor="black", arrowsize=2, font=dict(size=13))

# Percentage
fig.add_annotation(x=monthly_Washing_Machine_df.index[2], y=monthly_Shower_df['Flow'][5] + monthly_Shower_df['Flow'][5]*0.1, text=f"{round(np.abs((monthly_Shower_df['Flow'][8] - monthly_Washing_Machine_df['Flow'][1])/monthly_Shower_df['Flow'][8]), 2) * 100} %", showarrow=False,
                   arrowhead=1, arrowcolor="black", arrowsize=2, font=dict(size=15, color="red"))

fig.add_annotation(x=monthly_Kitchen_faucet_df.index[-2], y=monthly_Shower_df['Flow'][8] + monthly_Shower_df['Flow'][8]*0.1, text=f"{round(np.abs((monthly_Shower_df['Flow'][19] - monthly_Washing_Machine_df['Flow'][-3])/monthly_Shower_df['Flow'][8]), 2) * 100} %", showarrow=False,
                   arrowhead=1, arrowcolor="black", arrowsize=2, font=dict(size=15, color="red"))

# Customize the plot layout
fig.update_layout(
    title='Comparing Shower with other Fixtures',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

## 7. Comparing Flow Trends of Kitchen Faucet, Washbasin and Washing Machine

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Bidet_df.index, y=monthly_Bidet_df['Flow'], mode='lines+markers', name='Bidet'))
fig.add_trace(go.Scatter(x=monthly_Kitchen_faucet_df.index, y=monthly_Kitchen_faucet_df['Flow'], mode='lines+markers', name='Kitchen Faucet', line=dict(width=5)))
fig.add_trace(go.Scatter(x=monthly_Shower_df.index, y=monthly_Shower_df['Flow'], mode='lines+markers', name='Shower'))
fig.add_trace(go.Scatter(x=monthly_Washbasin_df.index, y=monthly_Washbasin_df['Flow'], mode='lines+markers', name='Washbasin', line=dict(width=5)))
fig.add_trace(go.Scatter(x=monthly_Washing_Machine_df.index, y=monthly_Washing_Machine_df['Flow'], mode='lines+markers', name='Washing Machine', line=dict(width=5)))
fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))

# Customize the plot layout
fig.update_layout(
    title='Comparing Flow Trends of Kitchen Faucet, Washbasin, and Washing Machine',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

## 8. Analysis of Kitchen Faucet Annually

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Bidet_df.index, y=monthly_Bidet_df['Flow'], mode='lines+markers', name='Bidet'))
fig.add_trace(go.Scatter(x=monthly_Kitchen_faucet_df.index, y=monthly_Kitchen_faucet_df['Flow'], mode='lines+markers', name='Kitchen Faucet', line=dict(width=4)))
fig.add_trace(go.Scatter(x=monthly_Shower_df.index, y=monthly_Shower_df['Flow'], mode='lines+markers', name='Shower'))
fig.add_trace(go.Scatter(x=monthly_Washing_Machine_df.index, y=monthly_Washing_Machine_df['Flow'], mode='lines+markers', name='Washing Machine'))
fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))

fig.add_trace(go.Scatter(x=[monthly_Kitchen_faucet_df.index[8]], y=[monthly_Kitchen_faucet_df['Flow'][8]], mode='markers', marker=dict(color='red', size=15), showlegend=False))
fig.add_trace(go.Scatter(x=[monthly_Kitchen_faucet_df.index[19]], y=[monthly_Kitchen_faucet_df['Flow'][19]], mode='markers', marker=dict(color='red', size=15), showlegend=False))

# Add a horizontal line
fig.add_trace(go.Scatter(x=[monthly_Kitchen_faucet_df.index[8], monthly_Kitchen_faucet_df.index[19]], y=[monthly_Kitchen_faucet_df['Flow'][8], monthly_Kitchen_faucet_df['Flow'][19]],
                         mode='lines', marker=dict(color='black', size=30), showlegend=False))

# Add text annotation
fig.add_annotation(x=monthly_Kitchen_faucet_df.index[8], y=monthly_Kitchen_faucet_df['Flow'][8], text=f"Oct 31, 2019", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

fig.add_annotation(x=monthly_Kitchen_faucet_df.index[19], y=monthly_Kitchen_faucet_df['Flow'][19], text=f"Sep 30, 2020", showarrow=True,
                   arrowhead=1, arrowcolor="black", arrowsize=2)

fig.add_annotation(x=monthly_Kitchen_faucet_df.index[13], y=monthly_Kitchen_faucet_df['Flow'][19] - monthly_Kitchen_faucet_df['Flow'][19]*0.1, text=f"{round(np.abs(((monthly_Kitchen_faucet_df['Flow'][8] - monthly_Kitchen_faucet_df['Flow'][19])/monthly_Kitchen_faucet_df['Flow'][8]*100)), 2)}%", showarrow=False,
                   arrowhead=1, arrowcolor="black", arrowsize=2, font=dict(size=15, color="red"), textangle=-10)

# Customize the plot layout
fig.update_layout(
    title='Analysis of Kitchen Faucet Annually',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

## 9. Analysis of Dishwasher & Toilet

In [ ]:
# Create a line plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=monthly_Dishwasher_df.index, y=monthly_Dishwasher_df['Flow'], mode='lines+markers', name='Dishwasher'))
fig.add_trace(go.Scatter(x=monthly_Toilet_df.index, y=monthly_Toilet_df['Flow'], mode='lines+markers', name='Toilet'))


# Customize the plot layout
fig.update_layout(
    title='Flow Trends of Dishwasher & Toilet Over Time',
    xaxis=dict(title='Datetime', tickangle=45),
    yaxis=dict(title='Flow'),
    hovermode='x unified',
    showlegend=True,
    template='plotly_white'
)

# Show the plot
fig.show()

In [ ]:
Dishwasher_count_monthly = Dishwasher_df.resample('M', on='end_datetime').count()
Toilet_count_monthly = Toilet_df.resample('M', on='end_datetime').count()

In [ ]:
Dishwasher_count_monthly = Dishwasher_count_monthly[Dishwasher_count_monthly['Flow'] > 0]
Toilet_count_monthly = Toilet_count_monthly[Toilet_count_monthly['Flow'] > 0]

In [ ]:
# Create a bar plot using Plotly
fig = go.Figure()

# Add a bar trace for the monthly count
fig.add_trace(go.Bar(x=Dishwasher_count_monthly.index.strftime('%Y-%m'), y=Dishwasher_count_monthly['Flow'], name='Count'))

# Customize the plot layout
fig.update_layout(
    title='Monthly Count of Dishwasher Flow Occurrences',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Count'),
    template='plotly_white'
)

# Show the plot
fig.show()

In [ ]:
# Create a bar plot using Plotly
fig = go.Figure()

# Add a bar trace for the monthly count
fig.add_trace(go.Bar(x=Toilet_count_monthly.index.strftime('%Y-%m'), y=Toilet_count_monthly['Flow'], name='Count'))

# Customize the plot layout
fig.update_layout(
    title='Monthly Count of Toilet Flow Occurrences',
    xaxis=dict(title='Month'),
    yaxis=dict(title='Count'),
    template='plotly_white'
)

# Show the plot
fig.show()